In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import ROOT
from ROOT import TFile, TTree, TH1F, TCanvas, TAxis, TLegend, TTreeReader, TTreeReaderValue
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
import multiprocessing
import time
import concurrent.futures

Welcome to JupyROOT 6.22/06


In [2]:
from joblib import dump, load
clf = load('BDT_model_1.joblib') 

In [3]:
def BDT_sort(event_record, evtnum, index, ismatched_arr):
    if not sum(evtnum) / 3 == evtnum[0]:
        print("Check if 3 candidates are coming from the same event")
        print(evtnum)
    best_score = 0
    best_index = -99
    score_BDT_temp = clf.predict(event_record)
#     print(score_BDT_temp)
#     print(index)
#     print(ismatched_arr)
    for i, score in enumerate(score_BDT_temp.ravel()):
        if score > best_score:
            best_score = score
            best_index = index[i]
#     print(best_score, best_index)
    return best_score, best_index

def select_trijet(sample, batch_size, ibatch, isSig):
    inFile = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
    inTree = inFile.Get("Events")
    variable2use = ['dijet_eta','dijet_phi','dR_jj','dEta_jj','dPhi_jj','dR_j0j2','dEta_j0j2','dPhi_j0j2','dR_j1j2','dEta_j1j2','dPhi_j1j2',
                    'jet_eta_0','jet_phi_0','jet_ptoverm_0','jet_eta_1','jet_phi_1','jet_ptoverm_1','jet_eta_2',
                    'jet_phi_2','jet_ptoverm_2','dR_jj_j','dEta_jj_j','dPhi_jj_j','jet_ptoverM_0','jet_ptoverM_1','jet_ptoverM_2',
                    'dijet_ptoverM']
    outFile = TFile(f"{sample}_BDT_Ranking_{ibatch}.root","RECREATE")
    outTree = TTree("Events","Events")
    
    dijet_eta = np.empty((1), dtype="float32")
    dijet_phi = np.empty((1), dtype="float32")
    dR_jj = np.empty((1), dtype="float32")
    dEta_jj = np.empty((1), dtype="float32")
    dPhi_jj = np.empty((1), dtype="float32")
    jet_eta_0 = np.empty((1), dtype="float32")
    jet_phi_0 = np.empty((1), dtype="float32")
    jet_ptoverm_0 = np.empty((1), dtype="float32")
    jet_eta_1 = np.empty((1), dtype="float32")
    jet_phi_1 = np.empty((1), dtype="float32")
    jet_ptoverm_1 = np.empty((1), dtype="float32")
    jet_eta_2 = np.empty((1), dtype="float32")
    jet_phi_2 = np.empty((1), dtype="float32")
    jet_ptoverm_2 = np.empty((1), dtype="float32")
    dR_jj_j = np.empty((1), dtype="float32")
    dEta_jj_j = np.empty((1), dtype="float32")
    dPhi_jj_j = np.empty((1), dtype="float32")
    jet_ptoverM_0 = np.empty((1), dtype="float32")
    jet_ptoverM_1 = np.empty((1), dtype="float32")
    jet_ptoverM_2 = np.empty((1), dtype="float32")
    dijet_ptoverM = np.empty((1), dtype="float32")
    M_jjj = np.empty((1), dtype="float32")
    m_jj = np.empty((1), dtype="float32")
    score_BDT = np.empty((1), dtype="float32")
    isMatched = np.empty((1), dtype="int32")

    outTree.Branch("dijet_eta", dijet_eta, "dijet_eta/F")
    outTree.Branch("dijet_phi", dijet_phi, "dijet_phi/F")
    outTree.Branch("dR_jj", dR_jj, "dR_jj/F")
    outTree.Branch("dEta_jj", dEta_jj, "dEta_jj/F")
    outTree.Branch("dPhi_jj", dPhi_jj, "dPhi_jj/F")
    outTree.Branch("jet_eta_0", jet_eta_0, "jet_eta_0/F")
    outTree.Branch("jet_phi_0", jet_phi_0, "jet_phi_0/F")
    outTree.Branch("jet_ptoverm_0", jet_ptoverm_0, "jet_ptoverm_0/F")
    outTree.Branch("jet_eta_1", jet_eta_1, "jet_eta_1/F")
    outTree.Branch("jet_phi_1", jet_phi_1, "jet_phi_1/F")
    outTree.Branch("jet_ptoverm_1", jet_ptoverm_1, "jet_ptoverm_1/F")
    outTree.Branch("jet_eta_2", jet_eta_2, "jet_eta_2/F")
    outTree.Branch("jet_phi_2", jet_phi_2, "jet_phi_2/F")
    outTree.Branch("jet_ptoverm_2", jet_ptoverm_2, "jet_ptoverm_2/F")
    outTree.Branch("dR_jj_j", dR_jj_j, "dR_jj_j/F")
    outTree.Branch("dEta_jj_j", dEta_jj_j, "dEta_jj_j/F")
    outTree.Branch("dPhi_jj_j", dPhi_jj_j, "dPhi_jj_j/F")
    outTree.Branch("jet_ptoverM_0", jet_ptoverM_0, "jet_ptoverM_0/F")
    outTree.Branch("jet_ptoverM_1", jet_ptoverM_1, "jet_ptoverM_1/F")
    outTree.Branch("jet_ptoverM_2", jet_ptoverM_2, "jet_ptoverM_2/F")
    outTree.Branch("dijet_ptoverM", dijet_ptoverM, "dijet_ptoverM/F")
    outTree.Branch("M_jjj", M_jjj, "M_jjj/F")
    outTree.Branch("m_jj", m_jj, "m_jj/F")
    outTree.Branch("score_BDT", score_BDT, "score_BDT/F")
    outTree.Branch("isMatched", isMatched, "isMatched/I")
    
    t_start = time.time()
    evtnum = []
    index = []
    event_record = []
    ismatched_arr = []
    evt_start = ibatch*batch_size
    evt_end = (ibatch+1)*batch_size
    if((ibatch+1)*batch_size > inTree.GetEntries()):
        evt_end = inTree.GetEntries()
    for ievt in range(evt_start, evt_end):
        inTree.GetEntry(ievt)
#         if ievt%1000 == 0:
#             print("Processing: ",ievt)
#             # time check:
#             t_end = time.time()
#             speed = (t_end - t_start)/(ievt+1)*1000
#             print(f"Avg. speed: {speed}s/1k candidates".format(speed))
#         if ievt > 100000: break

        if (ievt - evt_start)%100000 == 0:
            # time check:
            t_end = time.time()
            speed = (t_end - t_start)/(ievt - evt_start+1)*1000
            t_remain = (evt_end - ievt) / 1000 * speed / 60
            print(f"Batch #{ibatch} >> Avg. speed: {speed}s/1k candidates, time remaining: {t_remain}mins\n".format(ibatch, speed, t_remain))

        # make event record for BDT classification
        trijet_record = []
        for var in variable2use:
            trijet_record.append(getattr(inTree, var))
        if len(event_record) < 3:
            event_record.append(trijet_record)
            evtnum.append(inTree.evt_num)
            index.append(ievt)
            if isSig == 1:
                ismatched_arr.append(inTree.gen_dijet_matched)
            else:
                ismatched_arr.append(-99)
            if(len(event_record) == 3):
                best_score, best_index = BDT_sort(event_record, evtnum, index, ismatched_arr)
                inTree.GetEntry(best_index)
                dijet_eta[0] = inTree.dijet_eta
                dijet_phi[0] = inTree.dijet_phi
                dR_jj[0] = inTree.dR_jj
                dEta_jj[0] = inTree.dEta_jj
                dPhi_jj[0] = inTree.dPhi_jj
                jet_eta_0[0] = inTree.jet_eta_0
                jet_phi_0[0] = inTree.jet_phi_0
                jet_ptoverm_0[0] = inTree.jet_ptoverm_0
                jet_eta_1[0] = inTree.jet_eta_1
                jet_phi_1[0] = inTree.jet_phi_1
                jet_ptoverm_1[0] = inTree.jet_ptoverm_1
                jet_eta_2[0] = inTree.jet_eta_2
                jet_phi_2[0] = inTree.jet_phi_2
                jet_ptoverm_2[0] = inTree.jet_ptoverm_2
                dR_jj_j[0] = inTree.dR_jj_j
                dEta_jj_j[0] = inTree.dEta_jj_j
                dPhi_jj_j[0] = inTree.dPhi_jj_j
                jet_ptoverM_0[0] = inTree.jet_ptoverM_0
                jet_ptoverM_1[0] = inTree.jet_ptoverM_1
                jet_ptoverM_2[0] = inTree.jet_ptoverM_2
                dijet_ptoverM[0] = inTree.dijet_ptoverM
                M_jjj[0] = inTree.M_jjj
                m_jj[0] = inTree.m_jj
                score_BDT[0] = best_score
                if isSig == 1:
                    isMatched[0] = inTree.gen_dijet_matched
#                     print(isMatched[0])
                else:
                    isMatched[0] = -99
                outTree.Fill()
                event_record.clear()
                evtnum.clear()
                index.clear()
                ismatched_arr.clear()
#     print("Number of trijet candidates selected: ", outTree.GetEntries())
    outFile.cd()
    outTree.Write()
    outFile.Write()
    outFile.Close()
    print(f"Finished processing of batch {ibatch}/n")
    return 0

In [4]:
# QCD processing
if __name__ == '__main__':
    
#     for sample in ["QCD_Pt_470to600","QCD_Pt_600to800","QCD_Pt_800to1000","QCD_Pt_1000to1400","QCD_Pt_1400to1800","QCD_Pt_1800to2400",
#                    "QCD_Pt_2400to3200","QCD_Pt_3200toInf"]:
    for sample in ["QCD_Pt_800to1000","QCD_Pt_1000to1400","QCD_Pt_1400to1800","QCD_Pt_1800to2400",
                   "QCD_Pt_2400to3200","QCD_Pt_3200toInf"]:
        isSig = 0
        temp_file = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
        temp_tree = temp_file.Get("Events")
        tot_evts = temp_tree.GetEntries()
        print(tot_evts)
        expect_time = 0.3 # in hrs
        known_speed = 1.75 # sec per 1k candidates
        evt_batch = int(expect_time * 3600 / known_speed * 1000)
        if(evt_batch%3 != 0):
            print("!!!!")
        num_batch = math.ceil(tot_evts / evt_batch)
        print(f"Number of Candidates to be processed: {tot_evts}")
        print(f"Candidates to be processed per batch: {evt_batch}")
        print(f"Number of batches to be processed: {num_batch}")

        main_start = time.time()

    #     multiprocessing way
    #     p1 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_1))
    #     p2 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_2))
    #     p1.start()
    #     p1.join()
    #     p2.start()
    #     p2.join()
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = [executor.submit(select_trijet, sample, evt_batch, ibatch, isSig) for ibatch in range(num_batch)]
            status = [r.result() for r in results]
            print(status)

        print(f"Time used: {round(time.time() - main_start, 2)}")

2863974
Number of Candidates to be processed: 2863974
Candidates to be processed per batch: 617142
Number of batches to be processed: 5
Batch #0 >> Avg. speed: 20.51258087158203s/1k candidates, time remaining: 210.98625307083134mins

Batch #3 >> Avg. speed: 362.6437187194824s/1k candidates, time remaining: 3730.044497632981mins

Batch #1 >> Avg. speed: 374.12571907043457s/1k candidates, time remaining: 3848.144908642769mins

Batch #4 >> Avg. speed: 372.5695610046387s/1k candidates, time remaining: 2455.2706639766693mins

Batch #2 >> Avg. speed: 373.577356338501s/1k candidates, time remaining: 3842.50461409092mins

Batch #3 >> Avg. speed: 1.6203240915970358s/1k candidates, time remaining: 13.965627356277906mins

Batch #2 >> Avg. speed: 1.6365094136763099s/1k candidates, time remaining: 14.105129186789904mins

Batch #0 >> Avg. speed: 1.6499825560368664s/1k candidates, time remaining: 14.221254649900288mins

Batch #4 >> Avg. speed: 1.6531034049553304s/1k candidates, time remaining: 8.1389


Batch #2 >> Avg. speed: 1139.2343044281006s/1k candidates, time remaining: 2023.906703531742mins

Batch #1 >> Avg. speed: 1274.620532989502s/1k candidates, time remaining: 13110.364416170121mins

Batch #2 >> Avg. speed: 1.591075466980468s/1k candidates, time remaining: 0.17483267589670376mins

Batch #1 >> Avg. speed: 1.6003893148438817s/1k candidates, time remaining: 13.793808850949913mins

Batch #0 >> Avg. speed: 1.6103780903452256s/1k candidates, time remaining: 13.87990243995518mins

Finished processing of batch 2/n
Batch #1 >> Avg. speed: 1.5683217036509576s/1k candidates, time remaining: 10.903547535072795mins

Batch #0 >> Avg. speed: 1.5764768621644867s/1k candidates, time remaining: 10.960245187283638mins

Batch #1 >> Avg. speed: 1.5542749531255937s/1k candidates, time remaining: 8.21543111973595mins

Batch #0 >> Avg. speed: 1.5678937115487528s/1k candidates, time remaining: 8.287415791133244mins

Batch #1 >> Avg. speed: 1.5444151927549716s/1k candidates, time remaining: 5.5892

In [ ]:
# Signal processing
if __name__ == '__main__':
    
    sample = "Res1ToRes2GluTo3Glu_M1-3000_R-0p3"
    isSig = 1
    temp_file = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
    temp_tree = temp_file.Get("Events")
    tot_evts = temp_tree.GetEntries()
    
    expect_time = 2 # in mins
    known_speed = 1.8 # sec per 1k candidates
    evt_batch = int(expect_time * 60 / known_speed * 1000)
    num_batch = math.ceil(tot_evts / evt_batch)
    print(f"Number of Candidates to be processed: {tot_evts}")
    print(f"Candidates to be processed per batch: {evt_batch}")
    print(f"Number of batches to be processed: {num_batch}")
    
    main_start = time.time()
    
    batch_size = evt_batch
    
#     multiprocessing way
#     p1 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_1))
#     p2 = multiprocessing.Process(target=select_trijet, args=(sample, ibatch_2))
#     p1.start()
#     p1.join()
#     p2.start()
#     p2.join()

    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = [executor.submit(select_trijet, sample, batch_size, ibatch, isSig) for ibatch in range(num_batch)]
        status = [r.result() for r in results]
        print(status)

    print(f"Time used: {round(time.time() - main_start, 2)}")

In [ ]:
# Dry-run, speed calculation
sample = "QCD_Pt_300to470"
isSig = 0
temp_file = TFile(f"/home/xyan13/Trijet/TrijetAna/TrijetAna/outputs_3_jets/{sample}_ML_study.root","READ")
temp_tree = temp_file.Get("Events")
tot_evts = temp_tree.GetEntries()

main_start = time.time()

batch_size = evt_batch

select_trijet(sample, 100000, 0, isSig)

print(f"Time used: {round(time.time() - main_start, 2)}")